In [ ]:
import math
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import collections
from scipy.stats.stats import pearsonr
import pandas as pd
import os
from datetime import date

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

import seaborn as sns
import re

In [ ]:
# load of the data
DATASET_DIR = "dataset" + os.path.sep
#index_col=False say to not use the first column as ID
df_players = pd.read_csv('players.csv', sep=',', index_col=0) 

In [ ]:
df_players['ioc'] = df_players.ioc.astype("category").cat.codes

In [ ]:
df_players.info()

In [ ]:
df_prova = df_players[['best_rank', 'best_rank_points', 'best_of_3_match', 'best_of_5_match',
                      'best_of_3_wins(%)', 'best_of_5_wins(%)', 'num_wo_ret', 'num_def', 'tot_minutes',
                      'height','sv1st_win%', 'df%', 'ace_perc', 'bpS_perc']]

In [ ]:
df_prova = df_players[['best_rank', 'tot_minutes']]
df_prova = df_prova[~df_prova['tot_minutes'].isna()]

In [ ]:
scaler = StandardScaler()
scaler.fit(df_prova.values)

kmeans = KMeans(n_clusters=4, n_init=4, max_iter=10)
kmeans.fit(scaler.transform(df_prova.values))

np.unique(kmeans.labels_, return_counts=True)

plt.scatter(df_prova["best_rank"], df_prova["tot_minutes"], c=kmeans.labels_)
plt.show()

In [ ]:
df_prova = df_players[['best_rank', 'best_rank_points']]
df_prova = df_prova[(df_prova['best_rank']>0) & (df_prova['best_rank_points']>=0)]

scaler = StandardScaler()
scaler.fit(df_prova.values)

kmeans = KMeans(n_clusters=4, n_init=4, max_iter=10)
kmeans.fit(scaler.transform(df_prova.values))

np.unique(kmeans.labels_, return_counts=True)

plt.scatter(df_prova["best_rank"], df_prova["best_rank_points"], c=kmeans.labels_)
plt.show()

In [ ]:
f, axs = plt.subplots(nrows=6, ncols=6, figsize=(36,36))
plt.suptitle(("Visualization of the clustered data with respect to different feature spaces"), fontsize=28, fontweight='bold')
plot_id = 0
for i in range(len(cs_num.columns)):
    for j in range(i+1, len(cs_num.columns)):
        a, b = cs.columns[i], cs.columns[j]
        axs[int(plot_id/6)][plot_id%6].scatter(cs[a], cs[b], c=negative_sil_colors, s=20)
        for h in range(len(centers)):
            axs[int(plot_id/6)][plot_id%6].scatter(centers[h][cs.columns.get_loc(a)], centers[h][cs.columns.get_loc(b)], marker='o', c="white", alpha=1, s=200, edgecolor='k')
            axs[int(plot_id/6)][plot_id%6].scatter(centers[h][cs.columns.get_loc(a)], centers[h][cs.columns.get_loc(b)], marker='$%d$' % h, alpha=1, s=50, edgecolor='k')    
        axs[int(plot_id/6)][plot_id%6].set_title('Scatter( ' + a + ' , ' + b + ' ): K = ' + str(k), fontdict={'fontsize': 'x-large', 'fontweight' : 'bold'})
        axs[int(plot_id/6)][plot_id%6].set_xlabel(a, fontdict={'fontsize': 'x-large', 'fontweight' : 'bold'})
        axs[int(plot_id/6)][plot_id%6].set_ylabel(b, fontdict={'fontsize': 'x-large', 'fontweight' : 'bold'})
        plot_id = plot_id +1
plt.show()